<a href="https://colab.research.google.com/github/egitonga395/kamilimu_assignments/blob/sql/Ericksqlassignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Section 1
Use the athletes, regions, and sales datasets.


In [130]:
import sqlite3
import pandas as pd

# Load datasets
athletes = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-07-27/athlete_events.csv")
regions = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-07-27/noc_regions.csv")
sales = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/tips.csv")
sales = pd.read_csv("https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv", sep='\t')


# Connect to SQLite in-memory DB
conn = sqlite3.connect(":memory:")

# Write DataFrames to SQL tables
athletes.to_sql("athletes_table", conn, index=False, if_exists="replace")
regions.to_sql("regions_table", conn, index=False, if_exists="replace")
sales.to_sql("sales_table", conn, index=False, if_exists="replace")


4622

show the how the data looks like first

In [131]:
pd.read_sql("SELECT * FROM athletes_table LIMIT 5", conn)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,None
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,None
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,None
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,None


The athletes dataset:

**Q1**Count the total number of medals won by each country and show the top 5.



In [132]:
pd.read_sql("""
SELECT noc, COUNT(*) AS medal_count
FROM athletes_table
WHERE medal IS NOT NULL
GROUP BY noc
ORDER BY medal_count DESC
LIMIT 5
""", conn)

,NOC,medal_count
0,USA,5637
1,URS,2503
2,GER,2165
3,GBR,2068
4,FRA,1777


**Q2** Calculate the average age of athletes who won a Gold medal.

In [133]:
pd.read_sql_query("""
SELECT AVG(age) AS average_age
FROM athletes_table
WHERE medal = 'Gold'
""", conn)

,average_age
0,25.901013


**Q3** How many distinct events are there in each sport

In [134]:
pd.read_sql("""
SELECT sport, COUNT(DISTINCT event) AS distinct_events
FROM athletes_table
GROUP BY sport
ORDER BY COUNT(DISTINCT event) DESC
""", conn)

,Sport,distinct_events
0,Shooting,83
1,Athletics,83
2,Swimming,55
3,Cycling,44
4,Sailing,38
...,...,...
61,Cricket,1
62,Basque Pelota,1
63,Baseball,1
64,Alpinism,1


**Q4** Show all athletes from the United States (NOC = 'USA').


In [135]:
pd.read_sql("""
SELECT DISTINCT Name
FROM athletes_table
WHERE noc = 'USA'
""", conn)

,Name
0,Per Knut Aaland
1,John Aalberg
2,Stephen Anthony Abas
3,"David ""Dave"" Abbott"
4,Jeremy Abbott
...,...
9647,Frank Thomas Zuna
9648,David Santos Zuniga
9649,Rami Zur
9650,"Victor Andrew ""Vic"" Zwolak"


**Q5** Count how many medals were awarded each year.


In [136]:
pd.read_sql("""
SELECT year, COUNT(medal) AS medal_count
FROM athletes_table
WHERE medal IS NOT NULL
GROUP BY year
ORDER BY year
""", conn)

,Year,medal_count
0,1896,143
1,1900,604
2,1904,486
3,1906,458
4,1908,831
5,1912,941
6,1920,1308
7,1924,962
8,1928,823
9,1932,739



**Q6** Find all athlete records where height or weight is missing.



In [137]:
pd.read_sql("""
SELECT name AS athlete_name, height, weight
FROM athletes_table
WHERE height IS NULL OR weight IS NULL
""", conn)

,athlete_name,Height,Weight
0,Gunnar Nielsen Aaby,NaN,NaN
1,Edgar Lindenau Aabye,NaN,NaN
2,"Cornelia ""Cor"" Aalten (-Strannood)",168.0,NaN
3,"Cornelia ""Cor"" Aalten (-Strannood)",168.0,NaN
4,"Einar Ferdinand ""Einari"" Aalto",NaN,NaN
...,...,...,...
64258,Marius Edmund Zwiller,NaN,NaN
64259,Werner Zwingli,NaN,NaN
64260,Werner Zwingli,NaN,NaN
64261,Jan (Johann-) Zybert (Siebert-),NaN,NaN


**Q7** replacing the null value with average athelete weight or height

In [138]:
pd.read_sql("""
SELECT name AS athlete_name,
height, weight,
       CASE
           WHEN height IS NULL THEN (SELECT AVG(height) FROM athletes_table)
           ELSE height
       END AS height,
       CASE
           WHEN weight IS NULL THEN (SELECT AVG(weight) FROM athletes_table)
           ELSE weight
       END AS weight
FROM athletes_table
""", conn)

,athlete_name,Height,Weight,height,weight
0,A Dijiang,180.0,80.0,180.00000,80.000000
1,A Lamusi,170.0,60.0,170.00000,60.000000
2,Gunnar Nielsen Aaby,NaN,NaN,175.33897,70.702393
3,Edgar Lindenau Aabye,NaN,NaN,175.33897,70.702393
4,Christine Jacoba Aaftink,185.0,82.0,185.00000,82.000000
...,...,...,...,...,...
271111,Andrzej ya,179.0,89.0,179.00000,89.000000
271112,Piotr ya,176.0,59.0,176.00000,59.000000
271113,Piotr ya,176.0,59.0,176.00000,59.000000
271114,Tomasz Ireneusz ya,185.0,96.0,185.00000,96.000000


same question using coalesce

In [139]:
pd.read_sql("""
SELECT name AS athlete_name,
height, weight,
       COALESCE(height, (SELECT AVG(height) FROM athletes_table)) AS height,
       COALESCE(weight, (SELECT AVG(weight) FROM athletes_table)) AS weight
FROM athletes_table""", conn)

,athlete_name,Height,Weight,height,weight
0,A Dijiang,180.0,80.0,180.00000,80.000000
1,A Lamusi,170.0,60.0,170.00000,60.000000
2,Gunnar Nielsen Aaby,NaN,NaN,175.33897,70.702393
3,Edgar Lindenau Aabye,NaN,NaN,175.33897,70.702393
4,Christine Jacoba Aaftink,185.0,82.0,185.00000,82.000000
...,...,...,...,...,...
271111,Andrzej ya,179.0,89.0,179.00000,89.000000
271112,Piotr ya,176.0,59.0,176.00000,59.000000
271113,Piotr ya,176.0,59.0,176.00000,59.000000
271114,Tomasz Ireneusz ya,185.0,96.0,185.00000,96.000000



**Q8**
Return total sales value per item.



look at how the table looks like first

In [140]:
pd.read_sql("SELECT * FROM sales_table LIMIT 5", conn)

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,None,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,None,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


return total sale per item

In [141]:
pd.read_sql("""
PRAGMA table_info(sales_table);
""", conn)


,cid,name,type,notnull,dflt_value,pk
0,0,order_id,INTEGER,0,None,0
1,1,quantity,INTEGER,0,None,0
2,2,item_name,TEXT,0,None,0
3,3,choice_description,TEXT,0,None,0
4,4,item_price,TEXT,0,None,0


In [142]:
pd.read_sql("""
SELECT order_id, quantity, item_name, choice_description, item_price, CAST(REPLACE(item_price,'$','') AS float)* quantity AS total_sale
FROM sales_table
""", conn)

,order_id,quantity,item_name,choice_description,item_price,total_sale
0,1,1,Chips and Fresh Tomato Salsa,None,$2.39,2.39
1,1,1,Izze,[Clementine],$3.39,3.39
2,1,1,Nantucket Nectar,[Apple],$3.39,3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,None,$2.39,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98,33.96
...,...,...,...,...,...,...
4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",$11.75,11.75
4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",$11.75,11.75
4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",$11.25,11.25
4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",$8.75,8.75


**q9**
Show the top 5 records with the highest item_price.


top items with highest price

In [143]:
pd.read_sql("""
SELECT item_name, CAST(REPLACE(item_price,'$','') AS float) price
FROM sales_table
ORDER BY price DESC
LIMIT 5
""", conn)

,item_name,price
0,Chips and Fresh Tomato Salsa,44.25
1,Carnitas Bowl,35.25
2,Chicken Burrito,35.00
3,Chicken Burrito,35.00
4,Veggie Burrito,33.75


**Q10**
How many unique customer orders are there? (Assume each order_id is a customer.)

count distict customers from order ids

In [144]:
pd.read_sql("""
SELECT COUNT(DISTINCT order_id) AS distinct_customers
FROM sales_table
""", conn)

,distinct_customers
0,1834


Section 2

Write one query for this section.

Find countries with high-performing athletes in the Olympics. Use at least JOIN, NESTED QUERY, CASE, and optionally WITH.

For each country:

Count the number of athletes who won at least one medal.
Determine the average age of those medalists.
Create a new column called performance:
'High' if average age is below 25
'Medium' if between 25 and 30
'Low' if above 30


# **Thought process**


1. group by coutry
2. then count them
3. find the average age of medalist(with medal) is not none
4. classify them based on age

*    
*   grouby them by country
*   count ANY with one or more medals
*   find their age
*   then classify



Use the regions table for a join
Case for the classification














In [145]:
pd.read_sql("SELECT * FROM athletes_table LIMIT 5", conn)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,None
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,None
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,None
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,None


In [146]:
pd.read_sql("SELECT * FROM regions_table ", conn)

,NOC,region,notes
0,AFG,Afghanistan,None
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,None
3,ALG,Algeria,None
4,AND,Andorra,None
...,...,...,...
225,YEM,Yemen,None
226,YMD,Yemen,South Yemen
227,YUG,Serbia,Yugoslavia
228,ZAM,Zambia,None


In [156]:
pd.read_sql("""

SELECT region, COUNT(DISTINCT  Name) AS total_medalists, AVG(Age) AS average_age, Medal,
CASE
  WHEN AVG(Age) < 25 THEN 'High'
  WHEN AVG(Age) >= 25 AND AVG(Age) <=30 THEN 'Medium'
  WHEN AVG(Age) > 25 THEN 'Low'
END AS perfomance
FROM (SELECT regions_table.NOC, regions_table.region, athletes_table.Name, COALESCE(athletes_table.Age, (SELECT AVG(athletes_table.Age) FROM athletes_table)) AS Age, athletes_table.Medal
FROM athletes_table
INNER JOIN regions_table ON regions_table.NOC = athletes_table.NOC)
WHERE Medal IS NOT NULL
GROUP BY region
ORDER BY Name
""", conn)

,region,total_medalists,average_age,Medal,perfomance
0,Morocco,19,26.391304,Bronze,Medium
1,Niger,2,23.278449,Silver,High
2,Iraq,1,25.556898,Bronze,Medium
3,Ethiopia,31,25.547170,Bronze,Medium
4,Trinidad,23,23.864865,Bronze,High
...,...,...,...,...,...
131,Netherlands,731,26.383654,Bronze,Medium
132,Israel,8,24.444444,Silver,High
133,Colombia,22,26.285714,Bronze,Medium
134,Tajikistan,4,27.000000,Silver,Medium
